# Tomography as a Constrained Quadratic Problem on a Quantum Annealer

To run an optimization of a constrained quadratic model on DWaves Ocean platform, we have to initialize a quadratic model, optionally add constraints and then transfer the problem setting to the dedicated hybrid solver. We utilize numpy for classical preproccesing computations and solution calculation. To simulate a tomographic problem we utilize skimage to implement a Radon transform. To formulate our problem for the DWave system, we initialize a dimod ConstrainedQuadraticModel with dimod Integer variables. Subsequently, we use the LeapHybridCQMSampler to sample the solution on the DWave quantum annealer. As Ocean does not yet support matrix operations, we import and use Sympy [3] to formulate our problem description.

In [47]:
# Classical numerical library
import numpy as np
# Symbolic matrix computations
from sympy import sympify, Matrix
# Tomographic problem 
from skimage.transform import radon
# Constrained quadratic model
from dimod import ConstrainedQuadraticModel, Integer, Real, Binary
# Hybrid quantum annealing sampler
from dwave.system import LeapHybridCQMSampler

## Introduction to Tomography

Tomographic reconstruction is a multidimensional inverse problem, where the problem is to estimate an object by only having projection views of the object. In Computed Tomography the projection views are captured by placing an x-ray source on one part of the object and a detector on the other side. The x-rays are attenuated by the matter and captured by the detector. In Single Photon Emission Computed Tomography (SPECT) the situation changes as the radiation source is in the patient, therefore collimators are used in front of the detectors to capture parallel gamma rays. One projection view can be thought of as an integration/summation of the rotated object. The simple projection process and corresponding matrix notation is showcased below:

<p align="center">
 <img src="QuantumAnnealingTomography.PNG">
</p>

Here $Y$ represents our acquired projection values and $X$ is our object to be imaged. We use a zero-based number indexing for vectors and matrices to comply with Python defaults. We can write both $X$ and $Y$ in vector representation. The relationship between our object image $X$ and projections $Y$ can be described by the system matrix $M$. The system matrix $M$ is defined by the imaging geometry and describes each entry the conditional probability that an emission of $X_i$ was detected in $Y_j$.

$ Y = M * X =\begin{pmatrix} Y_0 \\ Y_1 \\ Y_2 \\ Y_3 \\ Y_4 \\ Y_5 \\ Y_6 \\ Y_7 \\ Y_8  \end{pmatrix} = M *  \begin{pmatrix} X_0 & X_1 & X_2 & X_3 & X_4 & X_5 & X_6 & X_7 & X_8  \end{pmatrix}^T  \; \; \; \; \; \text{ with } \; \; \; \; \; M =  p(detected \: in \: y  \: | emitted \: in \: x) = \begin{pmatrix} 1 & 1 & 1 & 0 & 0 & 0 & 0 & 0 & 0  \\
                 0 & 0 & 0 & 1 & 1 & 1 & 0 & 0 & 0  \\
                 1 & 1 & 1 & 0 & 0 & 0 & 1 & 1 & 1  \\
                 1 & 0 & 0 & 1 & 0 & 0 & 1 & 0 & 0  \\
                 0 & 1 & 0 & 0 & 1 & 0 & 0 & 1 & 0  \\
                 0 & 0 & 1 & 0 & 0 & 1 & 0 & 0 & 1  \\
 \end{pmatrix}
$

 The projection data can be obtained by the well known Radon transform. In the process of tomographic imaging, multiple views from different angles are captured to later reconstruct the object. The inverse problem rises in complexity when fewer views are chosen. In principal, the solution to the inverse tomographic problem is a matrix inversion of $M$ to solve for:
 
  $X = M^{-1}Y$ 

However, M in reality is sparse, very large, ill-posed, non-square and/or singular. Moreover, the projection values suffer from various source of noise e.g. Photon noise. Therefore, analytical methods like Filtered Backprojection (FBP) or iterative methods like Maximum Likelihood Expectation Maximization or Conjugate Gradient are used to reconstruct the original volume. Below, a realistic example of the imaged object and the corresponding projections is shown acquired with the Radon transform. 

<p align="center">
 <img src="Tomography.PNG">
</p>

## Create tomographic toy problem
1. Create the object to be imaged.

In [54]:
#object = np.array([[0, 0, 0, 0, 0], [0, 0, 5.0, 0, 0], [0, 5.0, 5.0, 5.0, 0], [0, 0, 5.0, 0, 0], [0, 0, 0, 0, 0]])
#object = np.array([[0, 0, 0, 0, 0], [0, 0, 0.0, 0, 0], [0, 5.0, 5.0, 5.0, 0], [0, 0, 0.0, 0, 0], [0, 0, 0, 0, 0]])
# object = np.array([[0, 0, 0, 0, 0, 0, 0], [0, 1.0, 1.0, 1.0, 1.0, 0.0, 0], [0, 1.0, 1.0, 1.0, 1.0, 1.0, 0], [0, 1.0, 1.0, 1.0, 1.0, 1.0, 0], [0, 1.0, 1.0, 0.0, 1.0, 1.0, 0], [0, 1.0, 1.0, 1.0, 1.0, 1.0, 0], [0, 0, 0, 0, 0, 0, 0]])
# object = np.array([[0, 0, 0], [0,5.0, 0], [0, 0, 0]])
object = np.array([[0, 0, 0, 0, 0, 0, 0], [0, 1.0, 0.0, 1.0, 0, 0.0, 0], [0, 0, 1.0, 0, 1.0, 0, 0], [0, 1.0, 0, 1.0, 0, 1.0, 0], [0, 1.0, 0, 0.0, 1.0, 1.0, 0], [0, 1.0, 0, 0, 0, 1.0, 0], [0, 0, 0, 0, 0, 0, 0]])
# object = np.array([[0, 0, 0, 0], [0, 3.0, 4.0, 0], [0, 5.0, 6.0, 0], [0, 0, 0, 0]])

2. Create the noise-free sinogram at equally spaced angles

In [55]:
# theta = np.linspace(0., 180., max(object.shape), endpoint=False)
theta = np.linspace(0., 180., max(object.shape), endpoint=False)
sinogram = radon(object, theta=theta)
sinogram.shape

(7, 7)

3. Create the noise-free system matrix

In [53]:
sinogram = sinogram.flatten()[..., np.newaxis]
object = object.flatten()[..., np.newaxis]
system = sinogram @ object.T
system = system / np.sum(object**2)
print(system[::2])

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [56]:
sinogram = sinogram.flatten()[..., np.newaxis]
object = object.flatten()[..., np.newaxis]

4. Create the noisy sinogram

In [ ]:
def createNoise(object, lowerBound=-1, upperBound=1):
    noise = np.random.randint(lowerBound, upperBound+1, size=object.shape)
    mask = object
    mask[mask != 0] = 1
    noise = mask * noise
    print(noise)
    return object + noise

In [ ]:
sinogram = np.zeros((max(object.shape), max(object.shape)))
theta = np.linspace(0., 360., max(object.shape), endpoint=False)
for i in range(max(object.shape)):
    noisy_object = createNoise(object)
    sinogram[..., i] = radon(noisy_object, theta=[theta[i]]).flatten()

## Matrix Inversion as a QUBO problem
This notebook is an extension and update to the implementation of the paper "Floating-Point Calculations on a Quantum Annealer: Division and Matrix Inversion" by M. Rogers and R. Singleton, 2020 [1]. 

The paper describes an approach to floating-point calculations on a quantum annealer. Specifically, they introduce a binary representation of the floating point variables. This representation is discrete, as they demonstrate their technique for 4 bit and 8 bit accuracy. With this measure they explain and derive the inversion of a matrix as a Quadratic Unconstrained Binary Optimization problem (QUBO), which is suitable to run on a DWave quantum annealer. 

We plan to expand the discrete (limited floating-point accuracy) to an integer problem solvable on the new hybrid solvers from DWave. In addition to solving integer style problems, the new hybrid solvers allow the integration of mathematical constraints. In the context of tomographic reconstruction for example non-negativity constraints are frequently used.

### Formulating the problem for the quantum annealer

The main challenge in formulating problems for quantum annealers is the description as a quadratic binary problem. Previously Rogers and Singleton have described matrix inversion as a quadratic binary problem for floating-point precision [1].   

Scalar notation of a QUBO problem: 

\begin{equation} E_{qubo}(a_i, b_{i,j}; q_i) = \sum_{i} a_i q_i + \sum_{i < j} b_{i, j} q_i q_j 
\end{equation} 


- $q_i$: Unknown binary variable (0, 1) we want to optimize for 
- $a_i$: Qubit bias/weight (real number)
- $b_{i,j}$: Coupling strength between spins $i$ and $j$ (real number)




Matrix (NxN) inversion is defined as: 

\begin{equation} M \cdot x = y \rightarrow x=M^{-1} \cdot y 
\end{equation} 

Formulate matrix inversion as a quadratic minimization problem with its minimum being the matrix inverse:

\begin{equation} H(x) = (Mx-y)^2 = \sum_{ijk=1}^{N} M_{ki} M_{kj} x^i x^j - 2 \sum_{ij=1}^{N} y_j M_{ji} x^i + \| y \|^2 = x^T M^T M x - x^T M^T y - y^t M x + y^T y
\end{equation} 

In matrix inversion and solving a linear system we usually solve for a vector of numbers which are typically not binary. Previously the common practice to obtain a floating point or integer representation of each component of x was to expand in powers of 2 multiplied by the boolean-valued variables $q_r^i \in \{0,1\}$:

\begin{equation} \chi^i = \sum_{r=0}^{R-1} 2^{-r} q_{r}^{i}
\end{equation} 

\begin{equation} x^i = 2\chi^i -1
\end{equation}

From the equation above we can see that the precision of the number heavily depends on the size or R (the number of bits used to represent the number). In the paper by Rogers and Singleton they opt for an R or 4, representing numbers as multiplies of 0.25 in the range of -1 to 3, this leads to a total number of 16 discrete values. 

The new constrained quadratic models (CQM) [2] running on DWaves Hybrid Samplers [3] enable the use of integer values in the quadratic model and therefore drastically expand solution possibilities. Further the possibility of introducing constraints is given. The new constrained quadratic model can be written as:

\begin{equation} H[q] = \sum_{i=1}^{N} a_i x_i + \sum_{i=1}^{N} \sum_{i \neq j}^{N}   b_{ij} x_i x_j  + c
\end{equation} 

- $x_i$: Unknown integer/binary variable we want to optimize for 
- $a_i$: Linear weight (real number)
- $b_{i,j}$: Quadratic term between $i$ and $j$ (real number)
- $c$: Inequality/equality constraints


## Implementation of the tomographic problem for the quantum annealer

### Problem formulation with sympy
We track our solution variables $x_i$ with the vector components. 

In [57]:
M = Matrix(system.tolist())
x_list = []
for i in range(object.shape[0]):
    x_list.append(sympify(str('x'+str(i))))
X = Matrix(x_list)
print(X)
Y = Matrix(sinogram.tolist())
M_C = system
Y_C = sinogram

Matrix([[x0], [x1], [x2], [x3], [x4], [x5], [x6], [x7], [x8], [x9], [x10], [x11], [x12], [x13], [x14], [x15], [x16], [x17], [x18], [x19], [x20], [x21], [x22], [x23], [x24], [x25], [x26], [x27], [x28], [x29], [x30], [x31], [x32], [x33], [x34], [x35], [x36], [x37], [x38], [x39], [x40], [x41], [x42], [x43], [x44], [x45], [x46], [x47], [x48]])


Reformulate the objective function into components of x as seen above: 


$H(x) = ||Mx-y||^2$

In [66]:
A = M.multiply(X)-Y
result = str(A.T.multiply(A)[0, 0])

In [73]:
# Make it compatible with lists
zero_entries = []
for i in reversed(range(len(X))):
    # print('x'+str(i))
    # print('x['+str(i)+']')
    tmp_res = result
    result = result.replace('x'+str(i)+' ', 'x['+str(i)+']')
    result = result.replace('x'+str(i)+')', 'x['+str(i)+'])')


Paste the above printed reformulation into cqm.set_objective(*printed reformulation*)

In [ ]:
#Constraints
lowerBound = 0
upperBound = 10
# Singular matrix case:
# upperBound = 6

#### Create the constrained quadratic model

In [76]:
# Create the quadratic model
cqm = ConstrainedQuadraticModel()
# Create a list of integer variables for each solution component
x = []
for i in range(len(X)):
    x.append(Binary('x'+str(i)))
# Reformulated objective function to minimize here
program = 'cqm.set_objective(' + result + ')'
print(result)
exec(program)
print(len(x))

0.00016450186287065*(x[10]+ x[11]+ x[15]+ x[16]+ x[17]+ x[18]+ x[19]+ x[22]+ x[23]+ x[24]+ x[25]+ x[26]+ x[29]+ x[30]+ x[32]+ x[33]+ x[36]+ x[37]+ x[38]+ x[39]+ x[40]+ x[8]+ x[9])**2 + 0.774329394293204*(0.0434782608695652*x[10]+ 0.0434782608695652*x[11]+ 0.0434782608695652*x[15]+ 0.0434782608695652*x[16]+ 0.0434782608695652*x[17]+ 0.0434782608695652*x[18]+ 0.0434782608695652*x[19]+ 0.0434782608695652*x[22]+ 0.0434782608695652*x[23]+ 0.0434782608695652*x[24]+ 0.0434782608695652*x[25]+ 0.0434782608695652*x[26]+ 0.0434782608695652*x[29]+ 0.0434782608695652*x[30]+ 0.0434782608695652*x[32]+ 0.0434782608695652*x[33]+ 0.0434782608695652*x[36]+ 0.0434782608695652*x[37]+ 0.0434782608695652*x[38]+ 0.0434782608695652*x[39]+ 0.0434782608695652*x[40]+ 0.0434782608695652*x[8]+ 0.0434782608695652*x[9]- 1)**2 + 0.774329394293201*(0.0434782608695652*x[10]+ 0.0434782608695652*x[11]+ 0.0434782608695652*x[15]+ 0.0434782608695652*x[16]+ 0.0434782608695652*x[17]+ 0.0434782608695652*x[18]+ 0.043478260869565

#### Run the constrained quadratic model on the hybrid sampler

In [77]:
# Sample from the Hybrid Solver on our constrained quadratic model
sampleset = LeapHybridCQMSampler().sample_cqm(cqm, label="Tomo Inversion Problem")
# Filter feasible solutions
feasible_sampleset = sampleset.filter(lambda row: row.is_feasible) 
# Print...
# print(sampleset)
print(feasible_sampleset) 

   x10 x11 x15 x16 x17 x18 x19 x22 x23 x24 x25 x26 ... x9    energy num_oc. ...
19 1.0 1.0 1.0 1.0 0.0 0.0 1.0 0.0 1.0 0.0 1.0 1.0 ... 0.0 16.629146       1 ...
25 1.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 1.0 0.0 0.0 1.0 ... 0.0 16.629146       1 ...
29 0.0 0.0 0.0 1.0 1.0 1.0 1.0 0.0 0.0 1.0 0.0 1.0 ... 0.0 16.629146       1 ...
31 1.0 1.0 1.0 1.0 0.0 0.0 0.0 1.0 0.0 1.0 1.0 0.0 ... 1.0 16.629146       1 ...
21 0.0 0.0 1.0 0.0 1.0 0.0 1.0 1.0 0.0 0.0 1.0 1.0 ... 1.0 16.629146       1 ...
23 1.0 1.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 1.0 ... 0.0 16.629146       1 ...
18 1.0 1.0 0.0 0.0 1.0 1.0 0.0 1.0 1.0 1.0 1.0 0.0 ... 1.0 16.629146       1 ...
16 1.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 1.0 0.0 ... 0.0 16.629146       1 ...
20 0.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0 1.0 1.0 1.0 0.0 ... 1.0 16.629146       1 ...
28 1.0 1.0 0.0 0.0 1.0 1.0 1.0 0.0 1.0 1.0 0.0 1.0 ... 0.0 16.629146       1 ...
17 1.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 1.0 0.0 ... 1.0 16.629146       1 ...
24 1.0 1.0 1.0 0.0 0.0 0.0 0.

In [71]:
import json
with open('sampleset_binary.json', 'w') as json_file:
    json.dump(feasible_sampleset.to_serializable(), json_file)

In [78]:
new_fes = feasible_sampleset

In [79]:
new_fes.first


Sample(sample={'x10': 1.0, 'x11': 1.0, 'x15': 1.0, 'x16': 1.0, 'x17': 0.0, 'x18': 0.0, 'x19': 1.0, 'x22': 0.0, 'x23': 1.0, 'x24': 0.0, 'x25': 1.0, 'x26': 1.0, 'x29': 0.0, 'x30': 0.0, 'x32': 1.0, 'x33': 1.0, 'x36': 0.0, 'x37': 0.0, 'x38': 1.0, 'x39': 0.0, 'x40': 1.0, 'x8': 0.0, 'x9': 0.0}, energy=16.629146004049232, num_occurrences=1, is_feasible=True, is_satisfied=array([], dtype=bool))

In [ ]:
object = np.array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
print(object.shape)

In [ ]:
sin_qa = system @ object
print(sin_qa.flatten())
print(sinogram.flatten())

### Classical solution
Solution using the classical inverse approximation.

In [ ]:
print("Classical solution: ", np.linalg.pinv(M_C).dot(Y_C))
print("Condition number: ", np.linalg.cond(M_C))

## References 
- [1] Rogers, Michael L., and Robert L. Singleton Jr. "Floating-point calculations on a quantum annealer: Division and matrix inversion." Frontiers in Physics 8 (2020): 265.
- [2] https://docs.ocean.dwavesys.com/en/latest/docs_dimod/reference/constrained.html
- [3] https://docs.ocean.dwavesys.com/projects/system/en/stable/reference/samplers.html
- [4] https://docs.sympy.org/latest/index.html